# Prototyp: Tokenizer

In [1]:
import regex
import pandas as pd

general = "01.01.2017, 01/01/2017, 1€, $2, £3,99, 4,99€, 5,99, 100, 9, U.S.A., AT&T, https://de.dariah.eu"
french = "Français! «Oui», \"m'éveillai\" étant \'Tchouang-tseu\'... Suis-je réalité? Qui rêve qu'il qui s'imagine!"
spanish = "Español! «Sí», \"de programación\" y años después del sueño léxico... \'Matemático meta-aplicado\'?"
english = "English! «Yes», it's the 2nd \"leaf-cutting\" ant at five o'clock... \'Didn't I say?\'"
portuguese = "Português! «Sim», \"ciência\" da computação é a ciência que estuda as técnicas... \'Classifica-se\' como ciência?"
german = "Deutsch! «Ja», wie geht's? \"Ein typisches Beispiel\" für ein System... OK, \'verbundenes-wort\'."

In [2]:
french_pattern = regex.compile(r'\p{L}[\p{L}\p{P}]*\p{L}|\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?')
print(french)

Français! «Oui», "m'éveillai" étant 'Tchouang-tseu'... Suis-je réalité? Qui rêve qu'il qui s'imagine!


In [3]:
pd.Series(french_pattern.findall(general))

0              01.01.2017
1              01/01/2017
2                   £3,99
3                    4,99
4                    5,99
5                     100
6                   U.S.A
7                    AT&T
8    https://de.dariah.eu
dtype: object

Nicht erfasst:
*  1€
*  $2
*  4,99€
*  9

In [4]:
pd.Series(french_pattern.findall(french))

0          Français
1               Oui
2        m'éveillai
3             étant
4     Tchouang-tseu
5           Suis-je
6           réalité
7               Qui
8              rêve
9             qu'il
10              qui
11        s'imagine
dtype: object

In [5]:
spanish_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}')
print(spanish)

Español! «Sí», "de programación" y años después del sueño léxico... 'Matemático meta-aplicado'?


In [6]:
pd.Series(spanish_pattern.findall(spanish))

0           Español
1                Sí
2                de
3      programación
4                 y
5              años
6           después
7               del
8             sueño
9            léxico
10       Matemático
11    meta-aplicado
dtype: object

In [7]:
english_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}|\p{N}\p{L}+')
print(english)

English! «Yes», it's the 2nd "leaf-cutting" ant at five o'clock... 'Didn't I say?'


In [8]:
pd.Series(english_pattern.findall(english))

0          English
1              Yes
2             it's
3              the
4              2nd
5     leaf-cutting
6              ant
7               at
8             five
9          o'clock
10          Didn't
11               I
12             say
dtype: object

In [9]:
portuguese_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}')
print(portuguese)

Português! «Sim», "ciência" da computação é a ciência que estuda as técnicas... 'Classifica-se' como ciência?


In [10]:
pd.Series(portuguese_pattern.findall(portuguese))

0         Português
1               Sim
2           ciência
3                da
4        computação
5                 é
6                 a
7           ciência
8               que
9            estuda
10               as
11         técnicas
12    Classifica-se
13             como
14          ciência
dtype: object

In [11]:
german_pattern = regex.compile(r'\p{L}[\p{L}\p{P}]*\p{L}|\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?')
print(german)

Deutsch! «Ja», wie geht's? "Ein typisches Beispiel" für ein System... OK, 'verbundenes-wort'.


In [12]:
pd.Series(german_pattern.findall(german))

0              Deutsch
1                   Ja
2                  wie
3               geht's
4                  Ein
5            typisches
6             Beispiel
7                  für
8                  ein
9               System
10                  OK
11    verbundenes-wort
dtype: object

## How it works

Jeder Buchstabe definiert eine bestimmte Kategorie:
*  `\p{L} or \p{Letter}: any kind of letter from any language.`
*  `\p{P} or \p{Punctuation}: any kind of punctuation character.`
*  `\p{N} or \p{Number}: any kind of numeric character in any script.`
*  `\p{S} or \p{Symbol}: math symbols, currency signs, dingbats, box-drawing characters, etc.`

Der englische Ausdruck näher betrachtet:
1. `\p{N}[\p{N}\p{P}]*\p{N}`
  *  Eine Zahl (`\p{N}`), oder keinmal oder mehrmals eine Zahl mit einem punctuation character (`[\p{N}\p{P}]*`) verbunden, gefolgt von einer Zahl (`\p{N}`).
    *  z. B. __01.01.2017__ oder __1999__
2. `\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?`
  * Keinmal oder einmal ein Symbol (`\p{S}?`), eine Zahl (`\p{N}`), ein punctuation character gefolgt von zwei Zahlen (`[\p{P}\p{N}]{3}`) und keinmal oder einmal ein Symbol (`\p{S}`).
    * z. B. __€5,99__
3. `\p{L}[\p{L}\p{P}]*\p{L}`
  * Ein Buchstabe (`\p{L}`), keinmal oder mehrmals ein Buchstabe gefolgt von einem punctuation character (`[\p{L}\p{P}]*`) und ein Buchstabe (`\p{L}`).
    * z. B. __Hello__ oder __leaf-cutting__
4. `\p{L}{1}`
  * Ein einzelner Buchstabe (wird mit 3. nicht gematched!).
    * z. B. __a__
5. `\p{N}\p{L}+`
  * Eine Zahl (`\p{N}`) und ein oder mehrere Buchstaben (`\p{L}+`).
    * z. B. __2nd__